현재 작업 디렉토리: /Users/sonmingi/Desktop/test_work/tms-routing/notebooks

현재 디렉토리 내용:
['03_tsp_solver.ipynb', '02_distance_matrix.ipynb', '01_fetch_coordinates.ipynb', '03_1_tsp_solver.ipynb']

Python 경로:
['/usr/local/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/usr/local/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/usr/local/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/sonmingi/Desktop/test_work/tms-routing/venv/lib/python3.10/site-packages', '/Users/sonmingi/Desktop/test_work/tms-routing/src', '/Users/sonmingi/Desktop/test_work/tms-routing/notebooks', '/Users/sonmingi/Desktop/test_work/tms-routing/notebooks', '/Users/sonmingi/Desktop/test_work/tms-routing/notebooks', '/Users/sonmingi/Desktop/test_work/tms-routing/notebooks', '/Users/sonmingi/Desktop/test_work/tms-routing/notebooks', '/Users/sonmingi/Desktop/test_work/tms-routing/notebook

In [1]:
import os
import sys

# 현재 경로에서 상위 디렉토리(프로젝트 루트)로 이동
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 기존 경로에서 중복된 항목들을 제거하고 프로젝트 루트를 맨 앞에 추가
sys.path = [project_root] + list(dict.fromkeys(sys.path))

import folium
import math
import numpy as np
from folium.map import Marker
from folium.features import DivIcon
from src.db.maria_client import fetch_orders
from src.model.distance import distance
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from geopy.distance import geodesic

In [5]:
df = fetch_orders()
points = [
    {
        "id": int(row["id"]),
        "latitude": row["latitude"],
        "longitude": row["longitude"],
        "address1": row["address1"],
        "address2": row["address2"],
    }
    for _, row in df.iterrows()
]

depots = [
    {"id": "depot_화성", "latitude": 37.157541, "longitude": 126.834845, "address1": "화성 허브센터", "address2": "", "demand": 0},
    {"id": "depot_이천", "latitude": 37.201687, "longitude": 127.459751, "address1": "이천 마장센터", "address2": "", "demand": 0},
    {"id": "depot_수원", "latitude": 37.263573, "longitude": 127.028601, "address1": "수원센터", "address2": "", "demand": 0},
    {"id": "depot_천안", "latitude": 36.815076, "longitude": 127.113818, "address1": "천안센터", "address2": "", "demand": 0},
    {"id": "depot_일산", "latitude": 37.705338, "longitude": 126.769166, "address1": "일산센터", "address2": "", "demand": 0},
    {"id": "depot_하남", "latitude": 37.535095, "longitude": 127.206303, "address1": "하남센터", "address2": "", "demand": 0},
    {"id": "depot_광주", "latitude": 37.413294, "longitude": 127.255902, "address1": "광주센터", "address2": "", "demand": 0},
    {"id": "depot_기흥", "latitude": 37.274234, "longitude": 127.115570, "address1": "기흥센터", "address2": "", "demand": 0}
]

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from geopy.distance import geodesic
import folium
from folium.map import Marker
from folium.features import DivIcon
import math
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from scipy.spatial import ConvexHull

def create_grid_regions(points, num_regions_lat, num_regions_lon):
    """지역을 격자로 분할"""
    lats = [p['latitude'] for p in points]
    lons = [p['longitude'] for p in points]
    min_lat, max_lat = min(lats), max(lats)
    min_lon, max_lon = min(lons), max(lons)
    
    lat_step = (max_lat - min_lat) / num_regions_lat
    lon_step = (max_lon - min_lon) / num_regions_lon
    
    grid = {}
    for point in points:
        lat_idx = int((point['latitude'] - min_lat) / lat_step)
        lon_idx = int((point['longitude'] - min_lon) / lon_step)
        lat_idx = min(lat_idx, num_regions_lat - 1)
        lon_idx = min(lon_idx, num_regions_lon - 1)
        
        grid_key = (lat_idx, lon_idx)
        if grid_key not in grid:
            grid[grid_key] = []
        grid[grid_key].append(point)
    
    return grid, (min_lat, max_lat, min_lon, max_lon)

def assign_grids_to_depots(grid, depots, bounds):
    """각 격자를 가장 가까운 depot에 할당"""
    depot_regions = {depot['id']: [] for depot in depots}
    
    for grid_key, points in grid.items():
        if not points:
            continue
        
        grid_center_lat = np.mean([p['latitude'] for p in points])
        grid_center_lon = np.mean([p['longitude'] for p in points])
        
        min_dist = float('inf')
        nearest_depot = None
        
        for depot in depots:
            dist = geodesic(
                (grid_center_lat, grid_center_lon),
                (depot['latitude'], depot['longitude'])
            ).km
            
            if dist < min_dist:
                min_dist = dist
                nearest_depot = depot
        
        depot_regions[nearest_depot['id']].extend(points)
    
    return depot_regions

def divide_region_into_vehicle_zones(points, num_vehicles, depot):
    """한 depot의 담당 구역을 차량 수만큼 분할"""
    if not points:
        return []
    
    point_angles = []
    for point in points:
        angle = math.atan2(
            point['latitude'] - depot['latitude'],
            point['longitude'] - depot['longitude']
        )
        point_angles.append((point, angle))
    
    point_angles.sort(key=lambda x: x[1])
    
    zones = [[] for _ in range(num_vehicles)]
    for i, (point, _) in enumerate(point_angles):
        zone_idx = (i * num_vehicles) // len(point_angles)
        zones[zone_idx].append(point)
    
    return zones

def optimize_vehicle_route(depot, points, vehicle_capacity, max_working_minutes=420, avg_speed_kmh=30):
    """단일 차량의 경로 최적화"""
    if not points:
        return None
    
    all_points = [depot] + points
    size = len(all_points)
    
    dist_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(i + 1, size):
            dist = geodesic(
                (all_points[i]['latitude'], all_points[i]['longitude']),
                (all_points[j]['latitude'], all_points[j]['longitude'])
            ).km
            dist_matrix[i][j] = dist * 1000
            dist_matrix[j][i] = dist * 1000
    
    manager = pywrapcp.RoutingIndexManager(size, 1, 0)
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(dist_matrix[from_node][to_node])
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    def time_callback(from_index, to_index):
        dist = distance_callback(from_index, to_index)
        travel_time = dist / 1000 * (60 / avg_speed_kmh)
        service_time = 7 if manager.IndexToNode(to_index) > 0 else 0
        return int(travel_time + service_time)
    
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(
        time_callback_index,
        15,
        max_working_minutes,
        True,
        'Time'
    )
    
    time_dimension = routing.GetDimensionOrDie('Time')
    time_dimension.SetGlobalSpanCostCoefficient(100)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 20
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if not solution:
        return None
    
    route = []
    route_distance = 0
    route_time = 0
    index = routing.Start(0)
    
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route.append(all_points[node_index])
        
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        
        if not routing.IsEnd(index):
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_time += time_callback(previous_index, index)
    
    return {
        'route': route,
        'distance': route_distance / 1000.0,
        'time': route_time
    }

def optimize_all_routes(points, depots, vehicle_capacity, num_regions_lat, num_regions_lon,
                       max_working_minutes=420, avg_speed_kmh=30):
    """전체 최적화 프로세스"""
    grid, bounds = create_grid_regions(points, num_regions_lat, num_regions_lon)
    depot_regions = assign_grids_to_depots(grid, depots, bounds)
    
    all_routes = []
    region_stats = []
    
    for depot in depots:
        region_points = depot_regions[depot['id']]
        if not region_points:
            continue
        
        print(f"\n처리 중: {depot['id']} (배송지점 수: {len(region_points)})")
        
        num_vehicles = math.ceil(len(region_points) / vehicle_capacity)
        vehicle_zones = divide_region_into_vehicle_zones(region_points, num_vehicles, depot)
        
        depot_routes = []
        vehicle_times = []
        
        for vehicle_id, zone_points in enumerate(vehicle_zones):
            if not zone_points:
                continue
                
            route_info = optimize_vehicle_route(
                depot, 
                zone_points, 
                vehicle_capacity,
                max_working_minutes=max_working_minutes,
                avg_speed_kmh=avg_speed_kmh
            )
            
            if route_info:
                route_info['vehicle_id'] = vehicle_id
                depot_routes.append(route_info)
                vehicle_times.append(route_info['time'])
        
        if depot_routes:
            all_routes.extend(depot_routes)
            
            total_distance = sum(r['distance'] for r in depot_routes)
            total_time = sum(r['time'] for r in depot_routes)
            time_efficiency = total_time / (len(depot_routes) * max_working_minutes)
            
            region_stats.append({
                'depot_id': depot['id'],
                'total_points': len(region_points),
                'total_vehicles': len(depot_routes),
                'total_distance': total_distance,
                'total_time': total_time,
                'avg_distance_per_vehicle': total_distance / len(depot_routes),
                'avg_time_per_vehicle': total_time / len(depot_routes),
                'time_efficiency': time_efficiency,
                'vehicle_times': vehicle_times
            })
    
    return all_routes, region_stats

def visualize_routes(routes, depots):
    """경로 시각화"""
    if not routes or not depots:
        return None
    
    center_lat = np.mean([d['latitude'] for d in depots])
    center_lon = np.mean([d['longitude'] for d in depots])
    m = folium.Map(location=[center_lat, center_lon], zoom_start=11)
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
              'lightblue', 'darkblue', 'darkgreen', 'cadetblue']
    
    for depot in depots:
        folium.Marker(
            location=[depot['latitude'], depot['longitude']],
            popup=f"Depot: {depot['id']}",
            icon=folium.Icon(color='black', icon='info-sign')
        ).add_to(m)
    
    for route_info in routes:
        color = colors[route_info['vehicle_id'] % len(colors)]
        route = route_info['route']
        
        folium.PolyLine(
            locations=[[p['latitude'], p['longitude']] for p in route],
            color=color,
            weight=2,
            opacity=0.8,
            popup=f"Distance: {route_info['distance']:.1f}km, Time: {route_info['time']:.0f}min"
        ).add_to(m)
        
        for i, point in enumerate(route[1:], 1):
            folium.CircleMarker(
                location=[point['latitude'], point['longitude']],
                radius=6,
                color=color,
                fill=True,
                popup=f"Stop {i}: {point.get('id', 'N/A')}"
            ).add_to(m)
    
    return m

# 실행
total_points = len(points)
print(f"총 배송지점 수: {total_points}")
print(f"총 depot 수: {len(depots)}")

lat_range = max(p['latitude'] for p in points) - min(p['latitude'] for p in points)
lon_range = max(p['longitude'] for p in points) - min(p['longitude'] for p in points)
aspect_ratio = lon_range / lat_range

optimal_points_per_grid = 10
base_grid_size = math.sqrt(total_points / optimal_points_per_grid)
num_regions_lat = max(5, int(base_grid_size))
num_regions_lon = max(5, int(base_grid_size * aspect_ratio))

print(f"\n최적화된 격자 분할: {num_regions_lat}x{num_regions_lon}")

vehicle_capacity = 35
max_working_hours = 420
average_speed = 30

optimized_routes, stats = optimize_all_routes(
    points=points,
    depots=depots,
    vehicle_capacity=vehicle_capacity,
    num_regions_lat=num_regions_lat,
    num_regions_lon=num_regions_lon,
    max_working_minutes=max_working_hours,
    avg_speed_kmh=average_speed
)

# 결과 분석 및 시각화
print("\n=== 최적화 결과 상세 분석 ===")
for stat in stats:
    print(f"\n[{stat['depot_id']}]")
    print(f"배송지점 수: {stat['total_points']:,}개")
    print(f"사용 차량 수: {stat['total_vehicles']}대")
    print(f"총 이동거리: {stat['total_distance']:.1f}km")
    print(f"총 소요시간: {stat['total_time']:.0f}분 ({stat['total_time']/60:.1f}시간)")
    print(f"차량당 평균 거리: {stat['avg_distance_per_vehicle']:.1f}km")
    print(f"차량당 평균 시간: {stat['avg_time_per_vehicle']:.0f}분")
    print(f"시간 효율성: {stat['time_efficiency']:.1%}")

m = visualize_routes(optimized_routes, depots)
display(m)

총 배송지점 수: 2015
총 depot 수: 8

최적화된 격자 분할: 14x10

처리 중: depot_화성 (배송지점 수: 114)

처리 중: depot_수원 (배송지점 수: 583)

처리 중: depot_천안 (배송지점 수: 70)

처리 중: depot_일산 (배송지점 수: 622)

처리 중: depot_하남 (배송지점 수: 241)
